<a href="https://colab.research.google.com/github/siddhusalvi/Pandas_DataFrame/blob/master/Working_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-s

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *

In [ ]:
df3 = spark.read.format("csv").option("header","true").load("day3.csv")

In [ ]:
df3.show()

+-------------------+---------+----------------+-------------+-----------+----------------+----------------------------------------+---------------------------------+-------------------------------+-----------------------+-----------------------+------------------------+-------------------+------------------+------------------+--------------------+----------------------+---------------------+--------------------+--------------------+----------------------+-----------------+----------------+----------------+---------------+----------------+---------------+----------------+----------------------------+--------------------------+----------------------------+--------------------+------------------------+----------------------+-------------------------+--------------------------------------+------------------------------------+---------------------------------------------------+---------------------------------------------------+--------------+--------------------+--------+------+----------

In [ ]:
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.types import *
from pyspark.sql import SQLContext
sqlContext = SQLContext
sc = spark.sparkContext
def addToRecord(record, name, time):
    if name in record.keys():
        time = time + record.get(name)
        record[name] = time
    else:
        record[name] = time
    return record

def getIdleTime(df):
    df1 = df.select('user_name', 'DateTime', 'keyboard', 'mouse') \
        .withColumn("Date", to_timestamp('DateTime', 'yyyy-MM-dd HH:mm:ss')) \
        .withColumnRenamed('user_name', "user") \
        .withColumnRenamed('Date', 'date') \
        .withColumn('action', col('keyboard').cast(IntegerType()) + col('mouse').cast(IntegerType())) \
        .drop('DateTime', 'keyboard', 'mouse') \
        .orderBy(col('user'), col('date'))
    users = list(df1.select('user').toPandas()['user'])
    action = list(df1.select('action').toPandas()['action'])

    name = ''
    counter = 0
    output = []
    record = {}

    def addToRecord(record, name, time):
        if name in record.keys():
            time = time * 5 + record.get(name)
            record[name] = time
        else:
            record[name] = time * 5
        return record

    for index in range(len(users)):
        if index == 0:
            if action[index] == 0:
                counter = 1
            name = users[index]
        elif index == (len(users) - 1):
            if ((users[index] != name) and counter > 5):
                addToRecord(record, name, counter)
            elif ((users[index] == name) and counter > 4 and action[index] == 0):
                counter = counter + 1
                addToRecord(record, name, counter)
        else:
            if (users[index] != name):
                if counter < 6:
                  counter = 0
                record = addToRecord(record, name, counter)
                if action[index] == 0:
                    counter = 1
                else:
                    counter = 0
                name = users[index]
            elif action[index] == 0:
                counter = counter + 1
            else:
                if counter > 5:
                    # print(name," ",counter)
                    record = addToRecord(record, name, counter)
                counter = 0
    lst = [(k, v) for k, v in record.items()]
    schema = StructType([
        StructField("user", StringType(), True),
        StructField("time", IntegerType(), True) ])

    newDF = spark.createDataFrame(lst, schema)
    return newDF  

In [ ]:
def getInOfficeTime(df):
  entryTimeDF =df.select('user_name','DateTime')\
  .withColumn("Date",to_timestamp('DateTime','yyyy-MM-dd HH:mm:ss'))\
  .withColumnRenamed('user_name',"user")\
  .withColumn('entryTime',col('Date'))\
  .orderBy('entryTime')\
  .drop_duplicates(['user'])\
  .drop('DateTime','Date')\
  .orderBy('entryTime')

  exitTimeDF = df.select('user_name','DateTime')\
  .withColumn("Date",to_timestamp('DateTime','yyyy-MM-dd HH:mm:ss'))\
  .withColumnRenamed('user_name',"user")\
  .withColumn('exitTime',col('Date'))\
  .sort(col("exitTime").desc())\
  .drop_duplicates(['user'])\
  .drop('DateTime','Date')

  df = entryTimeDF.join(exitTimeDF, on=['user'], how='inner')

  return df.withColumn("inOfficeTime", hour(col('exitTime')).cast('Integer')*60+minute(col('exitTime')).cast('Integer') - hour(col('entryTime')).cast('Integer')*60+minute(col('entryTime')).cast('Integer') )

In [ ]:
officetime1 = getInOfficeTime(df3)
officetime2 = getIdleTime(df3)

In [ ]:
df = officetime1.join(officetime2,on=['user'], how='outer')\
.withColumn('WorkingTime',col('inOfficeTime')-col('time'))
df.show()

+--------------------+-------------------+-------------------+------------+----+-----------+
|                user|          entryTime|           exitTime|inOfficeTime|time|WorkingTime|
+--------------------+-------------------+-------------------+------------+----+-----------+
|ashutoshrit64@gma...|2019-10-15 09:55:01|2019-10-15 19:30:01|         685|   0|        685|
|giridhardandikwar...|2019-10-15 09:20:01|2019-10-15 19:35:01|         655| 200|        455|
|       nikitapawar17|2019-10-15 09:25:02|2019-10-15 19:30:02|         655|   0|        655|
|umangsontakke70@g...|2019-10-15 09:25:01|2019-10-15 19:35:01|         660|  55|        605|
|indrajeetgajbhiye...|2019-10-15 09:00:01|2019-10-15 19:15:01|         615| 125|        490|
|khairnarswapna99@...|2019-10-15 14:10:01|2019-10-15 19:40:01|         350|   0|        350|
|gaikwadr576@gmail...|2019-10-15 09:35:01|2019-10-15 11:25:02|         180|   0|        180|
|adikumar2514@gmai...|2019-10-15 09:20:01|2019-10-15 19:20:02|        